### Transfer the pdf to text and dataframe

In [313]:
import pdftotext
import pandas as pd
import re

In [314]:
# Load your PDF
with open("/Users/timliu/Documents/on the desktop/MSc BA/論文/ARP/European (Re)Insurers/HNR1 GY/20150310_Hannover_Rueck_SE-_Earnings_Call_2015-3-10_FS000000002198853065.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

# Save all text to a txt file.
with open('/Users/timliu/Desktop/output/test.txt', 'w') as f:
    f.write("\n\n".join(pdf))

# open the text file
with open('/Users/timliu/Desktop/output/test.txt') as f:
    contents = f.readlines()

contents

# ToDo:
# write a function to processsing all the file

['Company N ame: H annover Rueck SE\n',
 'Company Ticker: H N R1 GR Equity\n',
 'Date: 2015-03-10\n',
 '\n',
 'Q4 2014 Earnings Call\n',
 'Company Participants\n',
 'FINAL\n',
 '\n',
 'Roland Vogel\n',
 'Ulrich Wallin\n',
 'Unidentified Participant\n',
 'Unverified Participant\n',
 '\n',
 'Other Participants\n',
 'Andrew J. Ritchie\n',
 'Ben Cohen\n',
 'Frank Kopfinger\n',
 'Janet Van den Berg\n',
 'Kamran Hossain\n',
 'Michael Haid\n',
 'Olivia S. Brindle\n',
 'Peter Casanova\n',
 'Rötger Franz\n',
 '\n',
 'Bloomberg Transcript\n',
 '\n',
 'Thomas Fossard\n',
 'Vikram Gandhi\n',
 '\n',
 'MANAGEMENT DISCUSSION SECTION\n',
 'Unidentified Participant\n',
 'Well, good afternoon to everybody here in London, and good afternoon depending on\n',
 "your time of day to all those who are joining us via the Internet. I wasn't going to give you\n",
 "a reminder about turning off the mobile phones, but it's easy to forget. So please do\n",
 'check if they are off.\n',
 "That said, welcome to Hannov

### Cleaning all the unwanted rows in the transcript

In [315]:
df = pd.DataFrame(contents)

# remove the unnessary string
df[0] = df[0].str.replace('\n','')
df[0] = df[0].str.replace('Bloomberg Transcript','')
df[0] = df[0].str.replace('\x0c\n','')
df[0] = df[0].str.replace('FINAL','')
df[0] = df[0].str.replace('A - ','')
df[0] = df[0].str.replace('Q - ','')

# using re to remove the unnessary string
def drop_unnessary(x):
    page = re.findall(r'Page \d+ of \d+', x) # 'page ... of ... '
    BIO = re.findall(r'{BIO', x) # '{BIO 18731996 <GO>}'
    Company_Name = re.findall(r'Company N ame:', x) # 'Company N ame: H annover Rueck SE'
    Company_Ticker = re.findall(r'Company Ticker:', x) # 'Company Ticker: H N R1 GR Equity'
    Date = re.findall(r'Date:', x) # Date: 2015-03-10
    if page == [] and BIO == [] and Company_Name == [] and Company_Ticker == [] and Date == []:
        return True
    else:
        return False

true_false = df[0].apply(lambda x: drop_unnessary(x))
df = df[true_false]

# drop the final page declaration
df = df[df[0] != 'This transcript may not be 100 percent accurate and may contain misspellings and other']
df = df[df[0] != 'inaccuracies. This transcript is provided "as is", without express or implied warranties of']
df = df[df[0] != 'any kind. Bloomberg retains all rights to this transcript and provides it solely for your']
df = df[df[0] != 'personal, non-commercial use. Bloomberg, its suppliers and third-party agents shall']
df = df[df[0] != 'have no liability for errors in this transcript or for lost profits, losses, or direct, indirect,']
df = df[df[0] != 'incidental, consequential, special or punitive damages in connection with the']
df = df[df[0] != 'furnishing, performance or use of such transcript. Neither the information nor any']
df = df[df[0] != 'opinion expressed in this transcript constitutes a solicitation of the purchase or sale of']
df = df[df[0] != 'securities or commodities. Any opinion expressed in the transcript does not necessarily']
# df = df[df[0] != 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights']  
df = df[df[0] != 'reserved. Any reproduction, redistribution or retransmission is expressly prohibited.']
# ¬© could not be identified, would apply re
def drop_Bloomberg_mark(x):
    Bloomberg_mark = re.findall(r'reflect the views of Bloomberg LP', x) # 'reflect the views of Bloomberg LP. ¬© COPYRIGHT 2022, BLOOMBERG LP. All rights'
    if Bloomberg_mark == []:
        return True
    else:
        return False

true_false = df[0].apply(lambda x: drop_Bloomberg_mark(x))
df = df[true_false]


# drop the empthy row
df = df[df[0] != '']
df = df[df[0] != '']


### Make a list of the participants in the meeting

In [316]:
# reset the index to make sure the index is continuous for better processing
df = df.reset_index(drop=True)

#  'Company Participants' index
df.loc[df[0] == 'Company Participants']
Participant_start_index = df.index[df[0] == 'Company Participants'].tolist()
#  'Other Participants' index
df.loc[df[0] == 'Other Participants']
Participant_middle_index = df.index[df[0] == 'Other Participants'].tolist()
#  'MANAGEMENT DISCUSSION SECTION' index, is the beginning of the management discussion, would stop before this row
df.loc[df[0] == 'MANAGEMENT DISCUSSION SECTION']
Participant_end_index = df.index[df[0] == 'MANAGEMENT DISCUSSION SECTION'].tolist()

print(Participant_start_index, Participant_middle_index, Participant_end_index)

# make the list of company_paticipants and other_participants
company_paticipants = df.loc[Participant_start_index[0]+1:Participant_middle_index[0]-1]
company_paticipants.drop(company_paticipants.index[company_paticipants[0] == ''].tolist(), inplace=True)

other_paticipants = df.loc[Participant_middle_index[0]+1:Participant_end_index[0]-1]
other_paticipants.drop(other_paticipants.index[other_paticipants[0] == ''].tolist(), inplace=True)

print("==========================")
print("the company paticipants is: ", company_paticipants)
print("==========================")
print("the other paticipants is: ", other_paticipants)

# after extract the paticipants, we can drop those information to make the transcript more clear
df = df.drop(range(df.index[df[0] == 'Company Participants'].tolist()[0],df.index[df[0] == 'MANAGEMENT DISCUSSION SECTION'].tolist()[0]+1))
# Q4 2014 Earnings Call
df.drop(df.index[df[0] == 'Q4 2014 Earnings Call'].tolist(), inplace=True)

# reset the index again to make sure the index is continuous for better processing
df = df.reset_index(drop=True)
# save to csv
df.to_csv('/Users/timliu/Desktop/output/df.csv')

[1] [6] [18]
the company paticipants is:                            0
2              Roland Vogel
3             Ulrich Wallin
4  Unidentified Participant
5    Unverified Participant
the other paticipants is:                       0
7    Andrew J. Ritchie
8            Ben Cohen
9      Frank Kopfinger
10  Janet Van den Berg
11      Kamran Hossain
12        Michael Haid
13   Olivia S. Brindle
14      Peter Casanova
15        Rötger Franz
16      Thomas Fossard
17       Vikram Gandhi


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Spilt the text into different df by different participants

In [317]:
# list of both company_paticipants and other_participants
# change the company_paticipants to list 
company_paticipants_list = company_paticipants[0].tolist()
other_paticipants_list = other_paticipants[0].tolist()
# merge the list
both_participants_list = company_paticipants_list + other_paticipants_list

In [318]:
# # create a dict to store all the data
# participants_dict = {}
# for name in both_participants_list:
#     participants_dict[name] = pd.DataFrame()
# participants_dict

# for i in range(len(both_participants_list)):
#     participants_1_start = df[df[0] == both_participants_list[i]].index
#     collect_index = []
#     for i in range(len(participants_1_start)):
#         collect_index.append(both_participants_row_index.index(participants_1_start[i]))
#         # #  add 1 into the collect_index
#     collect_index_end = [x+1 for x in collect_index]
#     # get the value of both_participants_row_index by collect_index_end
#     participants_1_end = [both_participants_row_index[x] for x in collect_index_end]
#     participants_1_end

In [319]:
# list of both company_paticipants and other_participants
# change the company_paticipants to list 
company_paticipants_list = company_paticipants[0].tolist()
other_paticipants_list = other_paticipants[0].tolist()
# merge the list
both_participants_list = company_paticipants_list + other_paticipants_list

# identify all the rows in df with both_participants in it
both_participants_row_index = df[df[0].isin(both_participants_list)].index.tolist()
both_participants_row_index

# identify all the rows in df with both_participants_list[0] in it
participants_1_start = df[df[0] == both_participants_list[0]].index
participants_1_start


collect_index = []
for i in range(len(participants_1_start)):
    collect_index.append(both_participants_row_index.index(participants_1_start[i]))

collect_index
#  add 1 into the collect_index
collect_index_end = [x+1 for x in collect_index]
collect_index, collect_index_end

# get the value of both_participants_row_index by collect_index_end
participants_1_end = [both_participants_row_index[x] for x in collect_index_end]
participants_1_end

# participants_1_start to list
participants_1_start = participants_1_start.tolist()
print(both_participants_list[0])
print(participants_1_start, participants_1_end)
print(len(participants_1_start), len(participants_1_end))

Roland Vogel
[86, 584, 659, 785, 803, 845, 902, 924, 930, 934] [329, 604, 672, 792, 805, 856, 914, 928, 932, 936]
10 10


In [320]:
for i in range(1,len(participants_1_start)):
    # participants_1 = df.loc[participants_1_start[0]+1:participants_1_end[0]-1]
    participants_1 = df.loc[participants_1_start[0]+1:participants_1_end[0]-1]
    participants_merge = df.loc[participants_1_start[i]+1:participants_1_end[i]-1]
    participants_1 = pd.concat([participants_1, participants_merge])

# concat all the rows into singl row
participants_1['value'] = 1
participants_1 = participants_1.groupby('value')[0].apply(''.join).reset_index()

# drop the "value" column
participants_1.drop(participants_1.columns[0], axis=1, inplace=True)

# rename the column
participants_1.rename(columns={0: both_participants_list[0]}, inplace=True)

# save to csv to check
participants_1.to_csv('/Users/timliu/Desktop/output/participants_1.csv')

participants_1


,Roland Vogel
0,"Yes, thank you, Uli. The good thing is that I ..."
